In [9]:
import pandas as pd
from fuzzywuzzy import fuzz
import string
import pickle as pkl

TRUE_CUTOFF = 1500

In [2]:
replace_char = ["*", "&", "%", "/", "\\"]
strip_char = ["'", "-", ".", "!", ":", ";"]
num_char =  ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
common_errors = {
    "0": "O",
    "1": "l",
    "5": "S",
    "8": "B"
}

In [3]:
map_dict = {}

In [4]:
def clean_header(h):
    for s in replace_char:
        h = h.replace(s, "")
    for s in strip_char:
        h = h.strip(s)
    cnt = 0
    hl = []
    for c in list(h):
        if c in common_errors: c = common_errors[c]
        if c in num_char: cnt += 1
        hl.append(c)
    h = string.join(hl, "")
    if cnt > 4: h = ""
    return h.upper()
    
def assign_clean(D):
    return [clean_header(h) for h in D.true_headers]

def score(string1, string2):
    # Scores the fuzzy match between the ocr header and the true header
    return fuzz.ratio(string1,string2)

def match(headers, true_headers, map_dict):
    # Matches ocr headers to true headers
    for header in headers:
        if header not in map_dict:
            score_list = [(score(header, true_header), true_header) for true_header in true_headers]
            sorted_score = sorted(score_list, key = lambda tup: tup[0], reverse=True)
            score_tuple = sorted_score[0]
            if score_tuple[0] > 90:
                print("LOG: Matched! " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = score_tuple
            else:
                print("LOG: Not matched, " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = "no_header"
    return map_dict

In [5]:
raw = pd.read_csv("RankedHeaders.csv")[["count", "true_headers"]].dropna().assign(true_headers=assign_clean)

In [6]:
df = raw.drop_duplicates("true_headers")
df = df[df.true_headers.map(lambda h: (len(h) < 150) and (len(h) > 2) and (h is not ""))].reset_index(drop=True)
true_headers = list(df[:TRUE_CUTOFF].true_headers)
unsure_headers = list(df[TRUE_CUTOFF:].true_headers)

In [7]:
map_dict = match(unsure_headers, true_headers, map_dict)

LOG: Matched! CLOTHING-RENTAL and CLOTHING-RETAIL with score 93
LOG: Not matched, MFRS A'GENTS and MIRS' AGENTS with score 83
LOG: Not matched, PAPER PRODUCTS MFRS and PLASTIC PRODUCTS- MFRS with score 78
LOG: Matched! STEEL AND STEEL PRODUCTS- DEALERS and STEEL AND STEEL PRODUCTS-DEALERS with score 98
LOG: Not matched, TL - SINLE I7SS and SCHOOL OF BUSINESS with score 55
LOG: Not matched, NEWS SERVICE BUREAUS and TRAVEL BUREAUS with score 65
LOG: Matched! DIAMONDS . PRECIOUS STONES and DIAMONDS  PRECIOUS STONES with score 98
LOG: Not matched, COD AV and WOOD AV with score 77
LOG: Matched! INTELLIGENCE OHICES  INTERIOR DECORATORS and INTELLIGENCE OFFICES  INTERIOR DECORATORS with score 96
LOG: Not matched, REAL ESTATE TITLES EXAMINED AND GUARANTEED and REAL ESTATE OWNERS AND DEALERS with score 61
LOG: Matched! C HIROPRAC TORS and CHIROPRACTORS with score 93
LOG: Not matched, GANSETT HOTEL and  2 HOTELS with score 55
LOG: Matched! CLOTHING-CHLLDREN'S AND INFANTS' -RETALL and CLOTHING-CH

LOG: Not matched, LAND OOMPANIES  and LAND COMPANIES with score 90
LOG: Matched! PALNT-RETAIL and PAINT-RETAIL with score 92
LOG: Not matched, ANGELONE and CHALKSTONE with score 56
LOG: Not matched,  LOANS and LO CRANSTO with score 63
LOG: Not matched, POUNDERS-IRON  STEEL and IRON  STEEL WORK with score 61
LOG: Not matched, FORESTERS and CORSETIERES with score 70
LOG: Matched! BUS  I(ROACH LINES-MOTOR and BUS  COACH LINES-MOTOR with score 91
LOG: Matched! OPTICAL GOODS- MANUFAC- TURERS and OPTICAL GOODS-MANUFAC- TURERS with score 98
LOG: Matched! DE LICATESSEN-RE TAIL and DELICATESSEN-RETAIL with score 95
LOG: Matched! GORSETIERES and CORSETIERES with score 91
LOG: Not matched, PIPE FITTINGS, HANGERS  VALVES and TYPEWRITING MACHINES-DEALERS with score 55
LOG: Not matched, AUTOMOBILE LIVERIES  AUTOMOBILE LOANS and AUTOMOBILE DEALERS-USED OARS with score 65
LOG: Not matched, AOL USTERS and ADJUSTERS with score 74
LOG: Not matched, GLASS MANUFACTURERS and PARTS MANUFACTURERS with score 8

LOG: Not matched, ENG INEERS-CLVIL and ENGINEERS-CIVIL with score 90
LOG: Matched! MUSICAL INSTRUMENT DEILERS 4 and MUSICAL INSTRUMENT DEALERS with score 93
LOG: Not matched, YARNS-MIRS and YARNS-MFRS with score 90
LOG: Not matched, MORTGAGES-REAL ESTATE and REAL ESTATE with score 69
LOG: Matched! AUC TIONEERS and AUCTIONEERS with score 96
LOG: Not matched, SAVINGS  LOAN ASSOCIATIONS and BUILDING  LOAN ASSOCIATIONS with score 83
LOG: Not matched, YAMS-AMH'S  WHOLESALE and YAMS-MFRS  WHOLESALE with score 88
LOG: Matched! TYPEWRITLNG MACHINES-DEALERS and TYPEWRITING MACHINES-DEALERS with score 96
LOG: Matched! CORSETIE'RES and CORSETIERES with score 96
LOG: Matched! E NGRAVERS-JEWE LRY and ENGRAVERS-JEWELRY with score 94
LOG: Not matched, STAMPING WORKS AND DIE and STAMPING WORKS AND DIE CUTTERS with score 85
LOG: Not matched, GOAL DEAIM-WHOLESALE and COAL DEALERS-WHOLESALE with score 81
LOG: Not matched, RADIO AND TELEVISION BROADCASTING COMPANIES AND STATIONS and RADIO BROADCASTING COM

LOG: Matched! SIGN PAINTERS  AND MFRS and SIGN PAINTERS AND MFRS with score 98
LOG: Not matched, TRUCKS- LEASING and TRUCKING with score 70
LOG: Not matched, RUBBER STAMPS AND SEALS and REAL ESTATE OWNERS AND DEALERS with score 60
LOG: Not matched, SHIPSMITHS  SHIRT MFRS and SHIRT MFRS with score 63
LOG: Not matched, HARDWARE-RE'I'AIL and HARDWARE-RETAIL with score 88
LOG: Matched! N EWSPAPERS and NEWSPAPERS with score 95
LOG: Matched! RETA IL and RETAIL with score 92
LOG: Matched! INSURANCE IGOMPANIES-LIABILITY and INSURANCE COMPANIES-LIABILITY with score 95
LOG: Matched! TEXTILE MLRS--YAR and TEXTILE MIRS-YAR with score 91
LOG: Matched! BAKERS-WHOIESALE and BAKERS-WHOLESALE with score 94
LOG: Not matched, SLIPPER MANUFACTURERS and SHIRT MANUFACTURERS with score 85
LOG: Matched! POULTRY DEALERSU-RETAIL and POULTRY DEALERS-RETAIL with score 98
LOG: Not matched, ICE CREAM DEALERS and ICE DEALERS with score 79
LOG: Not matched, FURNITURE LOANS and FURNITURE FINISHERS with score 71
LOG: M

LOG: Not matched, SPATIONE RS-R ETAIL and STATIONERS-RETAIL with score 89
LOG: Not matched, MUSIC AND MUSICAL INSTRU and MUSIC AND MUSICAL INSTRUMENT DEALERS with score 80
LOG: Not matched, JEWELRY MFRS- AVON JEWELRY MANUFACTURING and JEWELERS- MANUFACTURING with score 70
LOG: Not matched, LOANS-SIGNATURE and LOANS-MORTGAGE with score 62
LOG: Not matched,  HOS PITAIS and HOSPITALS with score 80
LOG: Not matched, MACHINISTB and MACHINISTS with score 90
LOG: Matched! BOTTLERS-LCARBONATEED BEVERAGES and BOTTLERS-CARBONATED BEVERAGES with score 97
LOG: Not matched, INSURANCE COMPANIES-CASUALTY and INSURANCE COMPANIES-LITE with score 85
LOG: Matched! REAL ESTATE INVESTMENTS and REAL ESTATE AND INVESTMENTS with score 92
LOG: Not matched, DVE STUFFS and DYE STUFFS with score 90
LOG: Matched! RETAI L and RETAIL with score 92
LOG: Not matched, HOPE 279 PLAINFIELD and SHOE REPAIRERS with score 48
LOG: Not matched, TRUCKS- MOTOR and TRUCK TIRES with score 67
LOG: Not matched, SELL and SELLS with 

...

LOG: Not matched, WOMEN'S GOWNS and EMPLOYMENT AGENCIES with score 50
LOG: Not matched, METER MFRS and JEWELRY MFRS with score 73
LOG: Not matched, DECORATORS-I'LAG  BUNTING and CONTRACTORS-HEATING with score 64
LOG: Not matched, BROKERS COMMODITIES and BROKERS-MONEY with score 63
LOG: Not matched, VEGETABLOS and VENETIAN BLINDS with score 64
LOG: Not matched, LLAUDITORS and LADIES' TAILORS with score 64
LOG: Not matched, BAGS- CELLOPHANE PLASTIC AND POLYETHELENE and SCHOOLS AND COLLEGE with score 50
LOG: Not matched, GAS  FILLING STATION EQUIPMENT and SUPPLIES AND EQUIPMENT with score 62
LOG: Not matched, FINANCING-COMMERCIAL and FINANCING-AUTOMOBILE with score 70
LOG: Not matched, CELLOPHANE GOODS-MRS and OPTICAL GOODS-MFRS with score 68
LOG: Not matched, SCREW MACHINE AND PRECBION PARTS MANU- FAC T URERS and MACHINERY MANUFACTURERS with score 60
LOG: Not matched,  PHOTOGRAPHERS-WEDDINGS and PHOTOGRAPHERS with score 72
LOG: Matched! ENTERTAINMENT BURE AUS and ENTERTAINMENT BUREAUS wi

LOG: Not matched, NIZ ED and VULCANIZERS with score 47
LOG: Not matched, STEEL PRODUCL'S-MFRS and PLASTIC PRODUCTS-MFRS with score 73
LOG: Not matched, ELECTRLC APPLIANCES and JEWELERSL SUPPLIES with score 65
LOG: Not matched, VIOLIN MAKERS AN D REPAIRERS and PIANO TUNERS AND REPAIRERS with score 74
LOG: Not matched, MAILING BOXES  TUBES and MAILING BOXES with score 79
LOG: Not matched, C'ONTRACTORS-CEMENT  GON- CRATE and CONTRACTORS-CONCRETE with score 75
LOG: Not matched, HOMES-I'INANCING and DRESS PLAITING AND TUCKING with score 52
LOG: Not matched, ENGINEERS-GHEMICAL and ENGINEERS-MECHANICAL with score 79
LOG: Not matched, MANU.FACTURLNG JEWELERS and MANUFACTURERS with score 72
LOG: Not matched, ADVERTISING--RADIO AND TELEVISION and RADIO AND TELEVISLON with score 72
LOG: Not matched, OHICE PARTITIONS and SERVICE STATIONS with score 63
LOG: Not matched, ME TAL GARAGES- MANUFACTURERS and METAL GOODS MANUFACTURERS with score 81
LOG: Not matched, LAWN MOWER REPAIRERS and SHOE REPAIRER

LOG: Not matched, PAPERHANUERS and PAPER RULERS with score 75
LOG: Not matched, LOANS- FARMS and YARNS-MFRS with score 64
LOG: Not matched, AUDLTORS and REALTORS with score 75
LOG: Not matched, PANN LANDS and PAN LES with score 71
LOG: Not matched, PACKING BOXES -WOODE and MAILING BOXES with score 61
LOG: Not matched, VENTILATING EQUIPMEN T and SUPPLIES AND EQUIPMENT with score 59
LOG: Not matched,  VARNISH DEALERS and  NEWS DEALERS with score 76
LOG: Not matched, CASH REGISTERS- DEALERS AND REPAIRERS and TIRE DEALERS AND REPAIRERS with score 76
LOG: Not matched, OILS-EDIBLE-MFRS and NOVELTIES-MFRS with score 60
LOG: Not matched, POUNDRY SUPPLIES AND EQUIP- MENT and SUPPLIES AND EQUIPMENT with score 81
LOG: Not matched, MANUFACTURERS-PIPE and MANUFACTURERS with score 84
LOG: Not matched, SOLAR HEATING and CONTRACTORS-HEATING with score 63
LOG: Not matched, GUTTERS AND OONDUCTOR PIPE and PAINTERS AND DECORATORS with score 57
LOG: Not matched, FFAMBULANCE SERVICE and SALES AND SERVICE wi

LOG: Not matched, I'OUNDRY SUPPLIES AND EQUIP- MENT and SUPPLIES AND EQUIPMENT with score 80
LOG: Not matched, RAW HIDE and LAW OFFICES with score 53
LOG: Not matched, ASNHALT SHINGLES AND ROOFING and ASPHALT SHINGLES with score 68
LOG: Not matched, WPHTHALMOLOGISTS and PSYCHOLOGISTS with score 69
LOG: Not matched, GATION 2OL WILLARD AV and ORGANIZATIONS- LABOR with score 49
LOG: Not matched, FLAVORIN G EXTRACTS AND ESSENCE MFRS and INTERIOR DECORATORS AND DESIGNERS with score 55
LOG: Not matched, PUM.IGATING  SPRDYING and PACKAGING, CRATING AND SHIPPING with score 58
LOG: Not matched, PINANCE AND LOANS and FINANCE COMPANIES with score 65
LOG: Not matched,  ONTBOACRD MOTORS and INTERIOR DE CORATORS with score 59
LOG: Not matched, WOVEN GOODS-COTTON and COTTON GOODS CONVERTERS with score 54
LOG: Not matched, EA 42S2 BARRINGTON PARKWAY EAST PROVIDENCE RADIO  ELECTRONIC EQUIPMENT and HOTEL AND RESTAURANT SUPPLIES AND EQUIPMENT with score 46
LOG: Not matched, MACHINISTS SUPPLIES AND TOOLS 

LOG: Not matched, WINDDWS-ALUMINUM ALL WEATHER and WINDOW CLEANERS with score 51
LOG: Not matched, LPREXTILE MACHINERY MFRS and MACHINERY MFRS with score 74
LOG: Not matched, CONTRACTORS-PALNTERS and CONTRACTORS-CARPENTER with score 83
LOG: Not matched, LIVESTOCK DEALERS AN D COMMISSION and TIRE DEALERS AND REPAIRING with score 61
LOG: Not matched, WOTTEE ROASTERS-WHOLESALE and GROCERS-WHOLESALE with score 71
LOG: Not matched, D-ALS AND MEDALLIONS and CANS AND PAILS with score 59
LOG: Not matched, BROIE RS- INVE STME NT and REAL ESTATE AND INVESTMENTS with score 61
LOG: Not matched, EXTRACT  ESSENCE MFRS  EYE EXAMINATIONS and CONTRACTORS-SEWER  DRAINS with score 50
LOG: Not matched, ALUMLNUM ALL WEATHER WIN- DOWS and VACUUM CLEANERS-DEALERS with score 49
LOG: Not matched, GESSPOOL CLEANING and PERSONAL LOANS with score 52
LOG: Not matched, ELECTRLCAL MFRS and ELECTRICAL CONTRACTORS with score 65
LOG: Not matched, CALENDARS-MANUFACTURERS and CASTINGS-MANUFACTURERS with score 80
LOG: Not

In [11]:
 pkl.dump(map_dict, open('trueheaders_match_dict.pkl', 'wb'))